In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve, make_scorer, f1_score, roc_auc_score
from sklearn import preprocessing
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, LeaveOneGroupOut, PredefinedSplit, GridSearchCV
import matplotlib.pyplot as plt
import os
import json

%matplotlib inline

In [2]:
def concat_dataframes(path, df_type):
    dfs_list = []
    dfs_rows_len_list = []
    
    for user in os.listdir(path):
        for file in os.listdir(os.path.join(path, user)):
            if file.find(df_type) != -1:
                df = pd.read_csv(os.path.join(path, user, file))
                
                df = df.drop(["timestamp"], axis=1)
                df = (df - df.min()) / (df.max() - df.min())
                
                df["user"] = int(user.split('_')[1])
                
                dfs_list.append(df)
    
    return pd.concat(dfs_list, ignore_index=True)


def drop_bad_rows(df):
    bad_rows = set()
    for col in df.columns:
        if col != "user":
            for user in df.user.unique():
                for x in list(
                    df.loc[df.user == user, :][
                        np.abs(stats.zscore(df.loc[df.user == user, col])) > 3
                    ].index
                ):
                    bad_rows.add(x)

            for x in list(df[col][np.abs(stats.zscore(df[col])) > 3].index):
                bad_rows.add(x)

    df.drop(list(bad_rows), axis=0, inplace=True)

    return df


def print_bad_cols(df):
    bad_cols = set()
    for col in df.columns:
        if col != "user":
            if df[df[col] != df[col].mean()].shape[0] < 0.1 * df.shape[0]:
                bad_cols.add(col)

            for user in df.user.unique():
                if (
                    df.loc[df.user == user, :][
                        df.loc[df.user == user, col]
                        != df.loc[df.user == user, col].mean()
                    ].shape[0]
                    < 0.1 * df.loc[df.user == user, :].shape[0]
                ):
                    bad_cols.add(col)

                elif (
                    np.sum(np.abs(stats.zscore(df.loc[df.user == user, col])) < 3)
                    < 0.9 * df.loc[df.user == user, col].shape[0]
                ):
                    bad_cols.add(col)

    print(list(bad_cols))
    
    
def extract_delayed_user(df, user_label):
    df_user = df[df["user"] == user_label]
    df = df[df["user"] != user_label]
    return df_user, df


def split_users_into_two_classes(df, valid_user_label):
    df.loc[df["user"] != valid_user_label, "user"] = 0
    df.loc[df["user"] == valid_user_label, "user"] = 1
    return df  


def get_cv_split(X, y, group_labels, valid_user_label):
    predefined_split_array = np.zeros(group_labels.shape[0])
    i = 0
    test_array = [x for x in range(group_labels.shape[0])]
    for test, _ in LeaveOneGroupOut().split(X, y, group_labels):
        diff = np.setdiff1d(test_array, test)
        if np.all(group_labels[diff[0] : diff[-1]] == valid_user_label) is np.bool_(True):
            for sample in diff:
                predefined_split_array[sample] = -1
        else:
            for sample in diff:
                predefined_split_array[sample] = i
            i += 1
    return predefined_split_array


def generate_train_dataset(df, user, ex_user, is_SVM = False):
    df_ = df.copy()

    df_for_test = []

    df__ = df_[df_.labels == ex_user].copy()
    df_for_test.append(df__)
    df_ = df_.drop(df__.index, axis=0)

    for user_ in df_.labels.unique():
        if user_ != ex_user:
            test_size = int((0.25 * df_[df_.labels == user_].shape[0]) - 1)
            df__ = df_[df_.labels == user_].sample(test_size).copy()
            df_for_test.append(df__)
            df_ = df_.drop(df__.index, axis=0)

    df_ = split_users_into_two_classes(df_.copy(), user)
          
    if is_SVM:    
        df_.loc[df_.user == 0, 'user'] = -1

    df_ = df_.drop("labels", axis=1)

    dataset = df_.to_numpy().copy()
    np.random.shuffle(dataset)

    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    return X, y, df_for_test


def generate_test_dataset(df_list, user, ex_user, is_SVM = False):
    test_df = pd.concat(df_list)

    valid_user_in_test_count = test_df[test_df.labels == user].shape[0]
    ex_user_in_test_count = test_df[test_df.labels == ex_user].shape[0]
    others_in_test_count = [test_df[test_df.labels == x].shape[0]
                            for x in test_df.labels.unique() if x != user and x != ex_user]

    others_test_count = sum(others_in_test_count)
    part_size = min(valid_user_in_test_count, ex_user_in_test_count)
    if others_test_count <= min(valid_user_in_test_count, ex_user_in_test_count):
        part_size = others_test_count    
        
    new_df_parts = []    

    new_df_parts.append(test_df[test_df.labels == user].sample(part_size).copy())
    new_df_parts.append(test_df[test_df.labels == ex_user].sample(part_size).copy())
    new_df_parts.append(test_df[~test_df.labels.isin([user, ex_user])].sample(part_size).copy())
    
    test_df = pd.concat(new_df_parts)
    
    test_df.loc[test_df.labels == user, "user"] = 1
    if is_SVM:
        test_df.loc[test_df.labels != user, "user"] = -1
    else:
        test_df.loc[test_df.labels != user, "user"] = 0

    print("True: ", test_df[test_df.user == 1].shape)
    print("Shape: ", test_df.shape)
    for x in test_df.labels.unique():
        print("Count ", x, ": ", test_df[test_df.labels == x].shape)

    test_df = test_df.drop("labels", axis=1)

    test_dataset = test_df.to_numpy().copy()
    X_test = test_dataset[:, :-1].copy()
    y_test = test_dataset[:, -1].copy()

    return X_test, y_test

def prepare_dataset(df, user, is_SVM=False):
    df_ = split_users_into_two_classes(df.copy(), user)
    
    group_labels = df_.labels.to_numpy().copy()
    df_ = df_.drop('labels', axis=1)
    
    if is_SVM:
        df_.loc[df_.user == 0, 'user'] = -1
    
    dataset = df_.to_numpy().copy()
    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    return X, y, group_labels


def create_file_for_results(data_type):
    res_folder = '.\\_results'
    if os.path.exists(res_folder) is False:
        os.makedirs(res_folder)
    
    file = os.path.join(res_folder, data_type + '_results.json')
    with open(file, 'w') as f:
        json.dump({'stub': None}, f)
        
    return file    


def update_file_with_results(file_path, results_dict):
    import collections.abc

    def update(d, u):
        for k, v in u.items():
            if isinstance(v, collections.abc.Mapping):
                d[k] = update(d.get(k, {}), v)
            else:
                d[k] = v
        return d
    
    with open(file_path, 'r') as f:
        res = json.load(f)
    
    res = update(res, results_dict)
    
    with open(file_path, 'w') as f:
        json.dump(res, f, sort_keys=True, indent=2)

In [3]:
df = concat_dataframes('..\\..\\scripts\\_features_all\\rolling\\60s', 'wifi')

In [4]:
df.columns.to_list()

['freq_mean',
 'freq_var',
 'freq_median',
 'freq_skew',
 'freq_kurt',
 'level_mean',
 'level_var',
 'level_median',
 'level_skew',
 'level_kurt',
 'count_mean',
 'count_var',
 'count_median',
 'count_skew',
 'count_kurt',
 'conn_level_mean',
 'conn_level_var',
 'conn_level_median',
 'conn_level_skew',
 'conn_level_kurt',
 'occured_nets_count_mean_mean',
 'occured_nets_count_mean_mad',
 'occured_nets_count_mean_skew',
 'occured_nets_count_median_mean',
 'occured_nets_count_median_mad',
 'occured_nets_count_median_skew',
 'occured_nets_count_var_mean',
 'occured_nets_count_var_mad',
 'occured_nets_count_var_skew',
 'disappeared_nets_count_mean_mean',
 'disappeared_nets_count_mean_mad',
 'disappeared_nets_count_mean_skew',
 'disappeared_nets_count_median_mean',
 'disappeared_nets_count_median_mad',
 'disappeared_nets_count_median_skew',
 'disappeared_nets_count_var_mean',
 'disappeared_nets_count_var_mad',
 'disappeared_nets_count_var_skew',
 'jaccard_index_mean_mean',
 'jaccard_index_me

In [5]:
features = [
 'freq_mean',
 'freq_var',
 'freq_median',
 'freq_skew',
 'freq_kurt',
 'level_mean',
 'level_var',
 'level_median',
 'level_skew',
 'level_kurt',
 'count_mean',
 'count_var',
 'count_median',
 'count_skew',
 'count_kurt',
 'conn_level_mean',
 'conn_level_var',
 'conn_level_median',
 'conn_level_skew',
 'conn_level_kurt',
 'occured_nets_count_mean_mean',
 'occured_nets_count_mean_mad',
 'occured_nets_count_mean_skew',
 'occured_nets_count_median_mean',
 'occured_nets_count_median_mad',
 'occured_nets_count_median_skew',
 'occured_nets_count_var_mean',
 'occured_nets_count_var_mad',
 'occured_nets_count_var_skew',
 'disappeared_nets_count_mean_mean',
 'disappeared_nets_count_mean_mad',
 'disappeared_nets_count_mean_skew',
 'disappeared_nets_count_median_mean',
 'disappeared_nets_count_median_mad',
 'disappeared_nets_count_median_skew',
 'disappeared_nets_count_var_mean',
 'disappeared_nets_count_var_mad',
 'disappeared_nets_count_var_skew',
 'jaccard_index_mean_mean',
 'jaccard_index_mean_mad',
 'jaccard_index_mean_skew',
 'jaccard_index_median_mean',
 'jaccard_index_median_mad',
 'jaccard_index_median_skew',
 'jaccard_index_var_mean',
 'jaccard_index_var_mad',
 'jaccard_index_var_skew',
 'occur_speed_mean_mean',
 'occur_speed_mean_mad',
 'occur_speed_mean_skew',
 'occur_speed_median_mean',
 'occur_speed_median_mad',
 'occur_speed_median_skew',
 'occur_speed_var_mean',
 'occur_speed_var_mad',
 'occur_speed_var_skew',
 'level_speed_mean_mean',
 'level_speed_mean_mad',
 'level_speed_mean_skew',
 'level_speed_median_mean',
 'level_speed_median_mad',
 'level_speed_median_skew',
 'level_speed_var_mean',
 'level_speed_var_mad',
 'level_speed_var_skew',
 'conn_level_speed_mean_mean',
 'conn_level_speed_mean_mad',
 'conn_level_speed_mean_skew',
 'conn_level_speed_median_mean',
 'conn_level_speed_median_mad',
 'conn_level_speed_median_skew',
 'conn_level_speed_var_mean',
 'conn_level_speed_var_mad',
 'conn_level_speed_var_skew',
 'count_speed_mean_mean',
 'count_speed_mean_mad',
 'count_speed_mean_skew',
 'count_speed_median_mean',
 'count_speed_median_mad',
 'count_speed_median_skew',
 'count_speed_var_mean',
 'count_speed_var_mad',
 'count_speed_var_skew',
 'user']

In [6]:
df = df.drop(df.columns.difference(features), axis=1)
df = df.dropna(how='all')
df = df.fillna(0)

In [ ]:
df = drop_bad_rows(df)
print_bad_cols(df)

D:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in greater
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in less


In [ ]:
# df = (df - df.min()) / (df.max() - df.min())

In [ ]:
# fig = plt.figure(figsize=(20, 25))
# plt.matshow(df.corr(), fignum=fig.number)
# plt.xticks(range(df.shape[1]), df.columns, fontsize=18, rotation=90)
# plt.yticks(range(df.shape[1]), df.columns, fontsize=18)
# cb = plt.colorbar()
# cb.ax.tick_params(labelsize=16)
# plt.title("Correlation matrix", fontsize=20, y=-0.03)

In [ ]:
corr_matrix = df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
corr_cols = [column for column in upper_tri.columns if any(abs(upper_tri[column]) > 0.7) and column != "user"]
df = df.drop(corr_cols, axis=1)

In [ ]:
# sample = df.sample(1000)
# plt.figure(figsize=(16, 10))
# plt.scatter(x=sample['conn_level_mean'], y=sample['count_var'], alpha=0.5, c=sample.user, cmap='magma')
# plt.colorbar()
# plt.show()

In [ ]:
df['labels'] = df['user']

In [ ]:
# for user in df.user.unique():
#     for valid_user in df.user.unique():
#         if user != valid_user:
#             print('---------------------------------------------------------------------------')
#             print('Valid user: ', valid_user, 'Extracted user: ', user)
#             print('---------------------------------------------------------------------------')
#             df1, df_ = extract_delayed_user(df.copy(), user)
#             df1['user'] = 0
#             df_ = split_users_into_two_classes(df_.copy(), valid_user)
#             df_ = resample(df_)

#             dataset = df_.to_numpy()
#             X = dataset[:, :-1]
#             y = dataset[:, -1]

#             X_test = df1.to_numpy()[:, :-1]
#             y_test = df1.to_numpy()[:, -1]
            
#             model = CatBoostClassifier(iterations=100, depth=6, loss_function='Logloss')
#             model.fit(X, y, verbose=False)

#             preds_class = model.predict(X_test)
#             print('Accuracy: ', accuracy_score(preds_class, y_test))

#             sum_ = 0
#             imp = [ (x, i) for x, i in zip(model.feature_importances_, range(len(model.feature_importances_)) )]
#             sorted_ = sorted(imp, key=lambda tup: tup[0])
#             for i in range(len(sorted_)):
#                 if sorted_[i][0] > 5:
#                     print(sorted_[i][1], ': ', df_.columns[sorted_[i][1]], ' - ', sorted_[i][0])

#             print('---------------------------------------------------------------------------')
#             print('---------------------------------------------------------------------------')

In [ ]:
# convert_to_json_key
def CJK(value):
    return "".join([str(value)])

In [ ]:
def model_cross_validation(model, df, model_tag, df_type):
    global RESULTS_FILE
    for user in df.labels.unique():
        print("Valid User: ", user)
        print("--------------------------------------------------------------------------------")

        X, y, group_labels = prepare_dataset(df, user)

        cv_split = PredefinedSplit(test_fold=get_cv_split(X, y, group_labels, user))
        scoring = ('accuracy')

        cv_results = cross_validate(model, X, y, scoring=scoring, cv=cv_split, n_jobs=-1)
        accuracy = cv_results['test_score']
        
        results = {}
        results[str(df_type)] = {}
        results[str(df_type)][str(model_tag)] = {}
        results[str(df_type)][str(model_tag)][str('CV')] = {}
        results[str(df_type)][str(model_tag)][str('CV')][str(user)] = {}
        results[str(df_type)][str(model_tag)][str('CV')][str(user)][str('accuracy')] = list(accuracy)
        results[str(df_type)][str(model_tag)][str('CV')][str(user)][str('mean_accuracy')] = float(np.mean(accuracy))
        results[str(df_type)][str(model_tag)][str('CV')][str(user)][str('max_accuracy')] = float(np.max(accuracy))
        results[str(df_type)][str(model_tag)][str('CV')][str(user)][str('min_accuracy')] = float(np.min(accuracy))
        
        update_file_with_results(RESULTS_FILE, results)

        print("CV accuracy list: ", accuracy)
        print("CV mean accuracy: ", np.mean(accuracy))
        print("CV min accuracy: ", np.min(accuracy))
        print("CV max accuracy: ", np.max(accuracy))

        print("--------------------------------------------------------------------------------")

In [ ]:
RESULTS_FILE = create_file_for_results('wifi')

## CatBoostClassifier CV

In [ ]:
model_params = {
    'iterations': 100,
    'depth': 6,
    'loss_function': 'Logloss',
    'l2_leaf_reg': 1,
    'leaf_estimation_iterations': 5,
    'logging_level': 'Silent'
}

CV_CATBOOST_BIG_DICT = {}
    
model = CatBoostClassifier(**model_params)
    
model_cross_validation(model, df, "CatBoost", "WIFI")

## CatBoostClassifier Final Validation

In [14]:
iterations = 100
depth = 6
loss_function = 'Logloss'
l2_leaf_reg = 1
leaf_estimation_iterations = 5
logging_level = 'Silent'

df["labels"] = df["user"]

VALIDATION_CATBOOST_BIG_DICT = {}

for user in df.labels.unique():
    print("Valid User: ", user)
    print("--------------------------------------------------------------------------------")
    VALIDATION_CATBOOST_BIG_DICT[str(user)] = {}

    for ex_user in df.labels.unique():
        if ex_user != user:

            X, y, df_for_test = generate_train_dataset(df, user, ex_user)

            model = CatBoostClassifier(iterations=iterations, depth=depth, loss_function=loss_function, \
                                       l2_leaf_reg=l2_leaf_reg, leaf_estimation_iterations=leaf_estimation_iterations, \
                                       logging_level=logging_level)
            model.fit(X, y)

            X_test, y_test = generate_test_dataset(df_for_test, user, ex_user)
            
            VALIDATION_CATBOOST_BIG_DICT[str(user)][ex_user] = {}
            VALIDATION_CATBOOST_BIG_DICT[str(user)][ex_user]["y_test"] = y_test.copy()
            VALIDATION_CATBOOST_BIG_DICT[str(user)][ex_user]["y_predict"] = model.predict(X_test).copy()
            VALIDATION_CATBOOST_BIG_DICT[str(user)][ex_user]["y_proba"] = model.predict_proba(X_test).copy()

            print("Valid user = ", user, ", Extracted user = ", ex_user, "accuracy = ",
                    accuracy_score(VALIDATION_CATBOOST_BIG_DICT[str(user)][ex_user]["y_test"], 
                                   VALIDATION_CATBOOST_BIG_DICT[str(user)][ex_user]["y_predict"])
                 )

            print("--------------------------------------------------------------------------------")

Valid User:  1
--------------------------------------------------------------------------------
True:  (1323, 25)
Shape:  (3969, 25)
Count  1 :  (1323, 25)
Count  2 :  (1323, 25)
Count  5 :  (919, 25)
Count  4 :  (383, 25)
Count  8 :  (9, 25)
Count  7 :  (5, 25)
Count  6 :  (4, 25)
Count  3 :  (3, 25)
Valid user =  1 , Extracted user =  2 accuracy =  0.999244142101285
--------------------------------------------------------------------------------
True:  (40, 25)
Shape:  (120, 25)
Count  1 :  (40, 25)
Count  3 :  (40, 25)
Count  5 :  (32, 25)
Count  2 :  (3, 25)
Count  4 :  (5, 25)
Valid user =  1 , Extracted user =  3 accuracy =  0.9833333333333333
--------------------------------------------------------------------------------
True:  (1323, 25)
Shape:  (3969, 25)
Count  1 :  (1323, 25)
Count  4 :  (1323, 25)
Count  5 :  (1150, 25)
Count  2 :  (134, 25)
Count  8 :  (17, 25)
Count  7 :  (18, 25)
Count  6 :  (3, 25)
Count  3 :  (1, 25)
Valid user =  1 , Extracted user =  4 accuracy =  0

True:  (51, 25)
Shape:  (153, 25)
Count  4 :  (51, 25)
Count  6 :  (51, 25)
Count  1 :  (12, 25)
Count  5 :  (31, 25)
Count  2 :  (8, 25)
Valid user =  4 , Extracted user =  6 accuracy =  0.9934640522875817
--------------------------------------------------------------------------------
True:  (197, 25)
Shape:  (591, 25)
Count  4 :  (197, 25)
Count  7 :  (197, 25)
Count  5 :  (130, 25)
Count  1 :  (47, 25)
Count  2 :  (17, 25)
Count  8 :  (2, 25)
Count  6 :  (1, 25)
Valid user =  4 , Extracted user =  7 accuracy =  0.9898477157360406
--------------------------------------------------------------------------------
True:  (255, 25)
Shape:  (765, 25)
Count  4 :  (255, 25)
Count  8 :  (255, 25)
Count  5 :  (180, 25)
Count  1 :  (52, 25)
Count  2 :  (18, 25)
Count  6 :  (1, 25)
Count  7 :  (4, 25)
Valid user =  4 , Extracted user =  8 accuracy =  0.9790849673202614
--------------------------------------------------------------------------------
Valid User:  5
-------------------------------

True:  (40, 25)
Shape:  (120, 25)
Count  8 :  (40, 25)
Count  3 :  (40, 25)
Count  4 :  (12, 25)
Count  5 :  (18, 25)
Count  1 :  (9, 25)
Count  2 :  (1, 25)
Valid user =  8 , Extracted user =  3 accuracy =  1.0
--------------------------------------------------------------------------------
True:  (62, 25)
Shape:  (186, 25)
Count  8 :  (62, 25)
Count  4 :  (62, 25)
Count  5 :  (40, 25)
Count  1 :  (14, 25)
Count  2 :  (8, 25)
Valid user =  8 , Extracted user =  4 accuracy =  1.0
--------------------------------------------------------------------------------
True:  (62, 25)
Shape:  (186, 25)
Count  8 :  (62, 25)
Count  5 :  (62, 25)
Count  2 :  (7, 25)
Count  4 :  (24, 25)
Count  1 :  (29, 25)
Count  7 :  (2, 25)
Valid user =  8 , Extracted user =  5 accuracy =  1.0
--------------------------------------------------------------------------------
True:  (51, 25)
Shape:  (153, 25)
Count  8 :  (51, 25)
Count  6 :  (51, 25)
Count  1 :  (10, 25)
Count  4 :  (11, 25)
Count  5 :  (25, 25)
Co

## RandomForestClassifier CV

In [15]:
n_estimators = 100
criterion = 'gini'
max_depth = None
min_samples_split = 2
min_samples_leaf = 1
max_features = 'auto'
n_jobs = -1
class_weight = 'balanced'

df['labels'] = df['user']

CV_RFC_BIG_DICT = {}

for user in df.labels.unique():
    print("Valid User: ", user)
    print("--------------------------------------------------------------------------------")
    
    X, y, group_labels = prepare_dataset(df, user)
    
    cv_split = PredefinedSplit(test_fold=get_cv_split(X, y, group_labels, user))
    scoring = ('accuracy')
    
    model = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, \
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, \
                                   max_features=max_features, n_jobs=n_jobs, class_weight=class_weight)
    
    cv_results = cross_validate(model, X, y, scoring=scoring, cv=cv_split, n_jobs=-1)
    accuracy = cv_results['test_score']
    
    CV_RFC_BIG_DICT[str(user)] = {}
    CV_RFC_BIG_DICT[str(user)]["accuracy"] = accuracy.copy()
    CV_RFC_BIG_DICT[str(user)]["mean_accuracy"] = np.mean(accuracy).copy()
    CV_RFC_BIG_DICT[str(user)]["max_accuracy"] = np.max(accuracy).copy()
    CV_RFC_BIG_DICT[str(user)]["min_accuracy"] = np.min(accuracy).copy()
    
    print("CV accuracy list: ", accuracy)
    print("CV mean accuracy: ", np.mean(accuracy))
    print("CV min accuracy: ", np.min(accuracy))
    print("CV max accuracy: ", np.max(accuracy))
    
    print("--------------------------------------------------------------------------------")

Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [1.         1.         0.99932387 0.80677945 1.         1.
 0.99215686]
CV mean accuracy:  0.9711800264208025
CV min accuracy:  0.8067794547224927
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [1.         1.         1.         1.         1.         1.
 0.83529412]
CV mean accuracy:  0.9764705882352941
CV min accuracy:  0.8352941176470589
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [1. 1. 1. 1. 1. 1. 1.]
CV mean accuracy:  1.0
CV min accuracy:  1.0
CV max accuracy:  1.0
-----------------------------------------------------------------

## RandomForestClassifier Final Validation

In [16]:
n_estimators = 100
criterion = 'gini'
max_depth = None
min_samples_split = 2
min_samples_leaf = 1
max_features = 'auto'
n_jobs = -1
class_weight = 'balanced'

df["labels"] = df["user"]

VALIDATION_RFC_BIG_DICT = {}

for user in df.labels.unique():
    print("Valid User: ", user)
    print("--------------------------------------------------------------------------------")
    
    VALIDATION_RFC_BIG_DICT[str(user)] = {}

    for ex_user in df.labels.unique():
        if ex_user != user:

            X, y, df_for_test = generate_train_dataset(df, user, ex_user)

            model = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, \
                                           min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, \
                                           max_features=max_features, n_jobs=n_jobs, class_weight=class_weight)
            model.fit(X, y)

            # Testing

            X_test, y_test = generate_test_dataset(df_for_test, user, ex_user)

            VALIDATION_RFC_BIG_DICT[str(user)][ex_user] = {}
            VALIDATION_RFC_BIG_DICT[str(user)][ex_user]["y_test"] = y_test.copy()
            VALIDATION_RFC_BIG_DICT[str(user)][ex_user]["y_predict"] = model.predict(X_test).copy()
            VALIDATION_RFC_BIG_DICT[str(user)][ex_user]["y_proba"] = model.predict_proba(X_test).copy()

            print("Valid user = ", user, ", Extracted user = ", ex_user, "accuracy = ",
                    accuracy_score(VALIDATION_RFC_BIG_DICT[str(user)][ex_user]["y_test"], 
                                   VALIDATION_RFC_BIG_DICT[str(user)][ex_user]["y_predict"])
                 )

            print("--------------------------------------------------------------------------------")

Valid User:  1
--------------------------------------------------------------------------------
True:  (1323, 25)
Shape:  (3969, 25)
Count  1 :  (1323, 25)
Count  2 :  (1323, 25)
Count  5 :  (901, 25)
Count  4 :  (393, 25)
Count  7 :  (12, 25)
Count  8 :  (10, 25)
Count  3 :  (3, 25)
Count  6 :  (4, 25)
Valid user =  1 , Extracted user =  2 accuracy =  0.9982363315696648
--------------------------------------------------------------------------------
True:  (40, 25)
Shape:  (120, 25)
Count  1 :  (40, 25)
Count  3 :  (40, 25)
Count  5 :  (27, 25)
Count  4 :  (10, 25)
Count  2 :  (3, 25)
Valid user =  1 , Extracted user =  3 accuracy =  1.0
--------------------------------------------------------------------------------
True:  (1323, 25)
Shape:  (3969, 25)
Count  1 :  (1323, 25)
Count  4 :  (1323, 25)
Count  5 :  (1166, 25)
Count  2 :  (128, 25)
Count  8 :  (11, 25)
Count  6 :  (3, 25)
Count  7 :  (13, 25)
Count  3 :  (2, 25)
Valid user =  1 , Extracted user =  4 accuracy =  0.9977324263

True:  (51, 25)
Shape:  (153, 25)
Count  4 :  (51, 25)
Count  6 :  (51, 25)
Count  5 :  (30, 25)
Count  2 :  (7, 25)
Count  1 :  (14, 25)
Valid user =  4 , Extracted user =  6 accuracy =  1.0
--------------------------------------------------------------------------------
True:  (197, 25)
Shape:  (591, 25)
Count  4 :  (197, 25)
Count  7 :  (197, 25)
Count  5 :  (131, 25)
Count  1 :  (51, 25)
Count  2 :  (12, 25)
Count  6 :  (1, 25)
Count  8 :  (2, 25)
Valid user =  4 , Extracted user =  7 accuracy =  0.9949238578680203
--------------------------------------------------------------------------------
True:  (255, 25)
Shape:  (765, 25)
Count  4 :  (255, 25)
Count  8 :  (255, 25)
Count  1 :  (58, 25)
Count  5 :  (182, 25)
Count  2 :  (14, 25)
Count  7 :  (1, 25)
Valid user =  4 , Extracted user =  8 accuracy =  1.0
--------------------------------------------------------------------------------
Valid User:  5
--------------------------------------------------------------------------------


True:  (40, 25)
Shape:  (120, 25)
Count  8 :  (40, 25)
Count  3 :  (40, 25)
Count  4 :  (17, 25)
Count  5 :  (15, 25)
Count  6 :  (1, 25)
Count  2 :  (4, 25)
Count  1 :  (3, 25)
Valid user =  8 , Extracted user =  3 accuracy =  0.9916666666666667
--------------------------------------------------------------------------------
True:  (62, 25)
Shape:  (186, 25)
Count  8 :  (62, 25)
Count  4 :  (62, 25)
Count  1 :  (17, 25)
Count  5 :  (42, 25)
Count  2 :  (3, 25)
Valid user =  8 , Extracted user =  4 accuracy =  0.989247311827957
--------------------------------------------------------------------------------
True:  (62, 25)
Shape:  (186, 25)
Count  8 :  (62, 25)
Count  5 :  (62, 25)
Count  4 :  (26, 25)
Count  2 :  (11, 25)
Count  1 :  (21, 25)
Count  7 :  (3, 25)
Count  3 :  (1, 25)
Valid user =  8 , Extracted user =  5 accuracy =  1.0
--------------------------------------------------------------------------------
True:  (51, 25)
Shape:  (153, 25)
Count  8 :  (51, 25)
Count  6 :  (51,

## SVC CV

In [17]:
C = 1
kernel = 'rbf'
degree = 1
gamma = 5

df['labels'] = df['user']

CV_SVC_BIG_DICT = {}

for user in df.labels.unique():
    print("Valid User: ", user)
    print("--------------------------------------------------------------------------------")
    
    X, y, group_labels = prepare_dataset(df, user, is_SVM=True)
    
    cv_split = PredefinedSplit(test_fold=get_cv_split(X, y, group_labels, user))
    scoring = ('accuracy')
    
    model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)
    
    cv_results = cross_validate(model, X, y, scoring=scoring, cv=cv_split, n_jobs=-1)
    accuracy = cv_results['test_score']
    
    CV_SVC_BIG_DICT[str(user)] = {}
    CV_SVC_BIG_DICT[str(user)]["accuracy"] = accuracy.copy()
    CV_SVC_BIG_DICT[str(user)]["mean_accuracy"] = np.mean(accuracy).copy()
    CV_SVC_BIG_DICT[str(user)]["max_accuracy"] = np.max(accuracy).copy()
    CV_SVC_BIG_DICT[str(user)]["min_accuracy"] = np.min(accuracy).copy()
    
    print("CV accuracy list: ", accuracy)
    print("CV mean accuracy: ", np.mean(accuracy))
    print("CV min accuracy: ", np.min(accuracy))
    print("CV max accuracy: ", np.max(accuracy))
    
    print("--------------------------------------------------------------------------------")

Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [0.9918284  0.975      0.99648411 0.45441821 1.         0.95431472
 0.9372549 ]
CV mean accuracy:  0.9013286197650547
CV min accuracy:  0.45441820837390456
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [0.99981118 1.         0.99878296 1.         1.         1.
 0.93333333]
CV mean accuracy:  0.9902753532916447
CV min accuracy:  0.9333333333333333
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [1.         0.99284985 1.         1.         0.17647059 0.87309645
 0.90588235]
CV mean accuracy:  0.8497570335227727
CV min accuracy:  0.17647058

## SVC Final Validation

In [18]:
C = 1
kernel = 'rbf'
degree = 1
gamma = 5

df["labels"] = df["user"]

VALIDATION_SVC_BIG_DICT = {}

for user in df.labels.unique():
    print("Valid User: ", user)
    print("--------------------------------------------------------------------------------")
    
    VALIDATION_SVC_BIG_DICT[str(user)] = {}

    for ex_user in df.labels.unique():
        if ex_user != user:

            X, y, df_for_test = generate_train_dataset(df, user, ex_user, is_SVM=True)

            model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)
            model.fit(X, y)

            # Testing

            X_test, y_test = generate_test_dataset(df_for_test, user, ex_user, is_SVM=True)

            VALIDATION_SVC_BIG_DICT[str(user)][ex_user] = {}
            VALIDATION_SVC_BIG_DICT[str(user)][ex_user]["y_test"] = y_test.copy()
            VALIDATION_SVC_BIG_DICT[str(user)][ex_user]["y_predict"] = model.predict(X_test).copy()
            VALIDATION_SVC_BIG_DICT[str(user)][ex_user]["y_proba"] = model.decision_function(X_test).copy()

            print("Valid user = ", user, ", Extracted user = ", ex_user, "accuracy = ",
                    accuracy_score(VALIDATION_SVC_BIG_DICT[str(user)][ex_user]["y_test"], 
                                   VALIDATION_SVC_BIG_DICT[str(user)][ex_user]["y_predict"])
                 )

            print("--------------------------------------------------------------------------------")

Valid User:  1
--------------------------------------------------------------------------------
True:  (1323, 25)
Shape:  (3969, 25)
Count  1 :  (1323, 25)
Count  2 :  (1323, 25)
Count  5 :  (892, 25)
Count  4 :  (399, 25)
Count  8 :  (16, 25)
Count  7 :  (9, 25)
Count  6 :  (4, 25)
Count  3 :  (3, 25)
Valid user =  1 , Extracted user =  2 accuracy =  0.9942050894431846
--------------------------------------------------------------------------------
True:  (40, 25)
Shape:  (120, 25)
Count  1 :  (40, 25)
Count  3 :  (40, 25)
Count  5 :  (23, 25)
Count  2 :  (5, 25)
Count  4 :  (11, 25)
Count  8 :  (1, 25)
Valid user =  1 , Extracted user =  3 accuracy =  0.9916666666666667
--------------------------------------------------------------------------------
True:  (1323, 25)
Shape:  (3969, 25)
Count  1 :  (1323, 25)
Count  4 :  (1323, 25)
Count  5 :  (1153, 25)
Count  2 :  (132, 25)
Count  8 :  (21, 25)
Count  7 :  (14, 25)
Count  3 :  (3, 25)
Valid user =  1 , Extracted user =  4 accuracy =

True:  (197, 25)
Shape:  (591, 25)
Count  4 :  (197, 25)
Count  7 :  (197, 25)
Count  5 :  (130, 25)
Count  1 :  (51, 25)
Count  2 :  (15, 25)
Count  8 :  (1, 25)
Valid user =  4 , Extracted user =  7 accuracy =  0.9780033840947546
--------------------------------------------------------------------------------
True:  (255, 25)
Shape:  (765, 25)
Count  4 :  (255, 25)
Count  8 :  (255, 25)
Count  1 :  (67, 25)
Count  5 :  (168, 25)
Count  2 :  (17, 25)
Count  7 :  (3, 25)
Valid user =  4 , Extracted user =  8 accuracy =  0.9882352941176471
--------------------------------------------------------------------------------
Valid User:  5
--------------------------------------------------------------------------------
True:  (2465, 25)
Shape:  (7395, 25)
Count  5 :  (2465, 25)
Count  1 :  (2465, 25)
Count  8 :  (62, 25)
Count  4 :  (1847, 25)
Count  2 :  (488, 25)
Count  7 :  (48, 25)
Count  6 :  (11, 25)
Count  3 :  (9, 25)
Valid user =  5 , Extracted user =  1 accuracy =  0.802974983096686

True:  (62, 25)
Shape:  (186, 25)
Count  8 :  (62, 25)
Count  4 :  (62, 25)
Count  5 :  (44, 25)
Count  1 :  (13, 25)
Count  2 :  (5, 25)
Valid user =  8 , Extracted user =  4 accuracy =  0.9731182795698925
--------------------------------------------------------------------------------
True:  (62, 25)
Shape:  (186, 25)
Count  8 :  (62, 25)
Count  5 :  (62, 25)
Count  4 :  (24, 25)
Count  2 :  (11, 25)
Count  1 :  (23, 25)
Count  7 :  (3, 25)
Count  6 :  (1, 25)
Valid user =  8 , Extracted user =  5 accuracy =  0.956989247311828
--------------------------------------------------------------------------------
True:  (51, 25)
Shape:  (153, 25)
Count  8 :  (51, 25)
Count  6 :  (51, 25)
Count  5 :  (27, 25)
Count  2 :  (4, 25)
Count  4 :  (11, 25)
Count  1 :  (8, 25)
Count  7 :  (1, 25)
Valid user =  8 , Extracted user =  6 accuracy =  0.9673202614379085
--------------------------------------------------------------------------------
True:  (62, 25)
Shape:  (186, 25)
Count  8 :  (62, 25)
C

## LogReg CV

In [19]:
penalty = 'l2'
C = 0.01
solver = 'newton-cg'
max_iter = 1000
n_jobs = -1

df['labels'] = df['user']

CV_LR_BIG_DICT = {}

for user in df.labels.unique():
    print("Valid User: ", user)
    print("--------------------------------------------------------------------------------")
    
    X, y, group_labels = prepare_dataset(df, user)
    
    cv_split = PredefinedSplit(test_fold=get_cv_split(X, y, group_labels, user))
    scoring = ('accuracy')
    
    model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=max_iter, n_jobs=n_jobs)
    
    cv_results = cross_validate(model, X, y, scoring=scoring, cv=cv_split, n_jobs=-1)
    accuracy = cv_results['test_score']
    
    CV_LR_BIG_DICT[str(user)] = {}
    CV_LR_BIG_DICT[str(user)]["accuracy"] = accuracy.copy()
    CV_LR_BIG_DICT[str(user)]["mean_accuracy"] = np.mean(accuracy).copy()
    CV_LR_BIG_DICT[str(user)]["max_accuracy"] = np.max(accuracy).copy()
    CV_LR_BIG_DICT[str(user)]["min_accuracy"] = np.min(accuracy).copy()
    
    print("CV accuracy list: ", accuracy)
    print("CV mean accuracy: ", np.mean(accuracy))
    print("CV min accuracy: ", np.min(accuracy))
    print("CV max accuracy: ", np.max(accuracy))
    
    print("--------------------------------------------------------------------------------")

Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [0.93309499 0.925      0.9979716  0.29789435 1.         0.96954315
 0.95294118]
CV mean accuracy:  0.8680636104983563
CV min accuracy:  0.2978943524829601
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [1. 1. 1. 1. 1. 1. 1.]
CV mean accuracy:  1.0
CV min accuracy:  1.0
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [1. 1. 1. 1. 1. 1. 1.]
CV mean accuracy:  1.0
CV min accuracy:  1.0
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  4
----------------------------------------

## LogReg Final Validation

In [20]:
penalty = 'l2'
C = 1
solver = 'newton-cg'
max_iter = 1000
n_jobs = -1

df["labels"] = df["user"]

VALIDATION_LR_BIG_DICT = {}

for user in df.labels.unique():
    print("Valid User: ", user)
    print("--------------------------------------------------------------------------------")
    
    VALIDATION_LR_BIG_DICT[str(user)] = {}

    for ex_user in df.labels.unique():
        if ex_user != user:

            X, y, df_for_test = generate_train_dataset(df, user, ex_user)
            
            model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=max_iter, n_jobs=n_jobs)
            model.fit(X, y)

            # Testing

            X_test, y_test = generate_test_dataset(df_for_test, user, ex_user)

            VALIDATION_LR_BIG_DICT[str(user)][ex_user] = {}
            VALIDATION_LR_BIG_DICT[str(user)][ex_user]["y_test"] = y_test.copy()
            VALIDATION_LR_BIG_DICT[str(user)][ex_user]["y_predict"] = model.predict(X_test).copy()
            VALIDATION_LR_BIG_DICT[str(user)][ex_user]["y_proba"] = model.predict_proba(X_test).copy()

            print("Valid user = ", user, ", Extracted user = ", ex_user, "accuracy = ",
                    accuracy_score(VALIDATION_LR_BIG_DICT[str(user)][ex_user]["y_test"], 
                                   VALIDATION_LR_BIG_DICT[str(user)][ex_user]["y_predict"])
                 )

            print("--------------------------------------------------------------------------------")

Valid User:  1
--------------------------------------------------------------------------------
True:  (1323, 25)
Shape:  (3969, 25)
Count  1 :  (1323, 25)
Count  2 :  (1323, 25)
Count  4 :  (366, 25)
Count  5 :  (923, 25)
Count  7 :  (9, 25)
Count  8 :  (19, 25)
Count  3 :  (3, 25)
Count  6 :  (3, 25)
Valid user =  1 , Extracted user =  2 accuracy =  0.9644746787603931
--------------------------------------------------------------------------------
True:  (40, 25)
Shape:  (120, 25)
Count  1 :  (40, 25)
Count  3 :  (40, 25)
Count  5 :  (23, 25)
Count  4 :  (14, 25)
Count  8 :  (1, 25)
Count  7 :  (1, 25)
Count  2 :  (1, 25)
Valid user =  1 , Extracted user =  3 accuracy =  0.9416666666666667
--------------------------------------------------------------------------------
True:  (1323, 25)
Shape:  (3969, 25)
Count  1 :  (1323, 25)
Count  4 :  (1323, 25)
Count  5 :  (1142, 25)
Count  2 :  (139, 25)
Count  6 :  (3, 25)
Count  8 :  (13, 25)
Count  7 :  (19, 25)
Count  3 :  (7, 25)
Valid us

True:  (51, 25)
Shape:  (153, 25)
Count  4 :  (51, 25)
Count  6 :  (51, 25)
Count  5 :  (39, 25)
Count  1 :  (9, 25)
Count  2 :  (3, 25)
Valid user =  4 , Extracted user =  6 accuracy =  0.869281045751634
--------------------------------------------------------------------------------
True:  (197, 25)
Shape:  (591, 25)
Count  4 :  (197, 25)
Count  7 :  (197, 25)
Count  5 :  (137, 25)
Count  1 :  (48, 25)
Count  2 :  (12, 25)
Valid user =  4 , Extracted user =  7 accuracy =  0.7918781725888325
--------------------------------------------------------------------------------
True:  (255, 25)
Shape:  (765, 25)
Count  4 :  (255, 25)
Count  8 :  (255, 25)
Count  5 :  (187, 25)
Count  1 :  (49, 25)
Count  7 :  (1, 25)
Count  2 :  (18, 25)
Valid user =  4 , Extracted user =  8 accuracy =  0.9006535947712418
--------------------------------------------------------------------------------
Valid User:  5
--------------------------------------------------------------------------------
True:  (2465

True:  (40, 25)
Shape:  (120, 25)
Count  8 :  (40, 25)
Count  3 :  (40, 25)
Count  4 :  (9, 25)
Count  1 :  (9, 25)
Count  5 :  (20, 25)
Count  2 :  (1, 25)
Count  7 :  (1, 25)
Valid user =  8 , Extracted user =  3 accuracy =  0.6833333333333333
--------------------------------------------------------------------------------
True:  (62, 25)
Shape:  (186, 25)
Count  8 :  (62, 25)
Count  4 :  (62, 25)
Count  5 :  (45, 25)
Count  1 :  (11, 25)
Count  2 :  (5, 25)
Count  7 :  (1, 25)
Valid user =  8 , Extracted user =  4 accuracy =  0.7204301075268817
--------------------------------------------------------------------------------
True:  (62, 25)
Shape:  (186, 25)
Count  8 :  (62, 25)
Count  5 :  (62, 25)
Count  4 :  (21, 25)
Count  2 :  (12, 25)
Count  7 :  (1, 25)
Count  1 :  (27, 25)
Count  6 :  (1, 25)
Valid user =  8 , Extracted user =  5 accuracy =  0.6720430107526881
--------------------------------------------------------------------------------
True:  (51, 25)
Shape:  (153, 25)
Co

In [21]:
for d in [CV_CATBOOST_BIG_DICT, CV_RFC_BIG_DICT, \
          CV_SVC_BIG_DICT, CV_LR_BIG_DICT]:
    
    for user, res in d.items():
        print("Valid User: ", user)
        print("--------------------------------------------------------------------------------")
        print('Mean accuracy: ', res['mean_accuracy'])

        print("--------------------------------------------------------------------------------")

Valid User:  1
--------------------------------------------------------------------------------
Mean accuracy:  0.9640620129407785
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
Mean accuracy:  0.9566978060259637
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
Mean accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  4
--------------------------------------------------------------------------------
Mean accuracy:  0.8449240391235285
--------------------------------------------------------------------------------
Valid User:  5
--------------------------------------------------------------------------------
Mean accuracy:  0.863351860501176
-------------------------------------

In [22]:
for d in [VALIDATION_CATBOOST_BIG_DICT, VALIDATION_RFC_BIG_DICT, \
          VALIDATION_SVC_BIG_DICT, VALIDATION_LR_BIG_DICT]:
    
    for user, res in d.items():
        print("Valid User: ", user)
        print("--------------------------------------------------------------------------------")
        means_acc = []
        means_prec = []
        means_rec = []
        means_roc = []
        means_f1 = []
        
        for ex_user, ex_res in res.items():
            print('Ex user: ', ex_user)
            print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

            y_true = ex_res['y_test']
            y_pred = ex_res['y_predict']
            if len(ex_res['y_proba'].shape) > 1 and ex_res['y_proba'].shape[1] > 1:
                y_proba = ex_res['y_proba'][:, 1]
            else:
                y_proba = ex_res['y_proba']
                
            acc = accuracy_score(y_true, y_pred)
            prec = precision_score(y_true, y_pred)
            rec = recall_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred)
            
            means_acc.append(acc)
            means_prec.append(prec)
            means_rec.append(rec)
            means_f1.append(f1)

            print('Accuracy: ', acc)
            print('Precision: ', prec)
            print('Recall: ', rec)
            try:
                roc = roc_auc_score(y_true, y_proba)
                means_roc.append(roc)
                print('ROC-AUC: ', roc)
            except Exception as e:
                print('ROC-AUC: skip')
            print('F1: ', f1)
            print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
            
        print('Mean accuracy: ', sum(means_acc) / len(means_acc))
        print('mean Precision: ', sum(means_prec) / len(means_prec))
        print('mean Recall: ', sum(means_rec) / len(means_rec))
        if len(means_roc) > 0:
            print('mean ROC-AUC: ', sum(means_roc) / len(means_roc))
        print('mean F1: ', sum(means_f1) / len(means_f1))

        print("--------------------------------------------------------------------------------")

Valid User:  1
--------------------------------------------------------------------------------
Ex user:  2
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.999244142101285
Precision:  0.9992435703479576
Recall:  0.99848828420257
ROC-AUC:  0.9999994286788371
F1:  0.9988657844990548
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  3
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.9833333333333333
Precision:  0.9523809523809523
Recall:  1.0
ROC-AUC:  0.999375
F1:  0.975609756097561
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  4
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.9977324263038548
Precision:  0.9969742813918305
Recall:  0.9962207105064248
ROC-AUC:  0.9998617402785419
F1:  0.9965973534971644
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  5
+++

Accuracy:  0.9989785495403473
Precision:  0.998467824310521
Recall:  0.998467824310521
ROC-AUC:  0.9999943919344019
F1:  0.998467824310521
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  3
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  1.0
Precision:  1.0
Recall:  1.0
ROC-AUC:  1.0
F1:  1.0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  4
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.9646230465395844
Precision:  0.904052165812762
Recall:  1.0
ROC-AUC:  0.9990229545948606
F1:  0.9496086105675147
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  6
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  1.0
Precision:  1.0
Recall:  1.0
ROC-AUC:  1.0
F1:  1.0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  7
+++++++++++++++++++++

Accuracy:  0.9377676996724615
Precision:  0.8435504469987228
Recall:  0.99848828420257
ROC-AUC:  0.9992727081594375
F1:  0.9145032883350639
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  6
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  1.0
Precision:  1.0
Recall:  1.0
ROC-AUC:  1.0
F1:  1.0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  7
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.9983079526226735
Precision:  1.0
Recall:  0.9949238578680203
ROC-AUC:  1.0
F1:  0.9974554707379135
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  8
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.9947712418300654
Precision:  0.9921568627450981
Recall:  0.9921568627450981
ROC-AUC:  0.9999307958477509
F1:  0.9921568627450981
+++++++++++++++++++++++++++++++++++++++++++++++

Accuracy:  1.0
Precision:  1.0
Recall:  1.0
ROC-AUC:  1.0
F1:  1.0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  8
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.9934640522875817
Precision:  0.9807692307692307
Recall:  1.0
ROC-AUC:  1.0
F1:  0.9902912621359222
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Mean accuracy:  0.9601133825298546
mean Precision:  0.9232196883399705
mean Recall:  0.9998534394884012
mean ROC-AUC:  0.999921513024982
mean F1:  0.9535222913132296
--------------------------------------------------------------------------------
Valid User:  6
--------------------------------------------------------------------------------
Ex user:  1
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  1.0
Precision:  1.0
Recall:  1.0
ROC-AUC:  1.0
F1:  1.0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user

Accuracy:  0.9993169398907104
Precision:  1.0
Recall:  0.9979508196721312
ROC-AUC:  0.9999979004299919
F1:  0.9989743589743589
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  6
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  1.0
Precision:  1.0
Recall:  1.0
ROC-AUC:  1.0
F1:  1.0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  7
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  1.0
Precision:  1.0
Recall:  1.0
ROC-AUC:  1.0
F1:  1.0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  8
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.9777777777777777
Precision:  0.9407407407407408
Recall:  0.996078431372549
ROC-AUC:  0.9998154555940023
F1:  0.9676190476190477
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Mean accuracy:  0.996727816809

...

D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Pr

 0.9937731007920774
F1:  0.9575458778416872
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  2
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.8476809240209349
Precision:  0.7000398883127243
Recall:  0.9501894964807797
ROC-AUC:  0.9269587727534283
F1:  0.8061552595314653
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  3
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.8666666666666667
Precision:  0.7307692307692307
Recall:  0.95
ROC-AUC:  0.975
F1:  0.8260869565217392
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  5
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.9658906334596643
Precision:  0.9363157894736842
Recall:  0.963183540877098
ROC-AUC:  0.9928031203529099
F1:  0.9495596477181745
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Accuracy:  0.6881720430107527
Precision:  1.0
Recall:  0.06451612903225806
ROC-AUC:  0.9681321540062435
F1:  0.12121212121212122
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  2
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.7311827956989247
Precision:  0.7727272727272727
Recall:  0.27419354838709675
ROC-AUC:  0.8350676378772112
F1:  0.4047619047619047
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  3
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.6833333333333333
Precision:  1.0
Recall:  0.05
ROC-AUC:  

D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8934375000000001
F1:  0.09523809523809523
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  4
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.7204301075268817
Precision:  1.0
Recall:  0.16129032258064516
ROC-AUC:  0.9155827263267429
F1:  0.27777777777777773
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  5
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.6720430107526881
Precision:  1.0
Recall:  0.016129032258064516
ROC-AUC:  0.9940166493236212
F1:  0.031746031746031744
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ex user:  6
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Accuracy:  0.673202614379085
Precision:  1.0
Recall:  0.0196078431372549
ROC-AUC:  0.9234909650134564
F1:  0.038461538461538464
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
